### The JSON outline of the research outline follows this structure:

{ 
   "title": "1. Introduction",
   "subsections": [
       {
           "title": "1.1 The Global Food Challenge and the Role of Irrigation",
           "point1": {
               "text": "...",
               "query1": "...",
               "query2": "...",
               ...
           },
           "point2": {
               "text": "...", 
               "query1": "...",
               "query2": "...",
               ...
           },
           ...
       },
       ...
   ]
}

#### The mapping from JSON to Excel is as follows:

1. The `point_text` value itself, mapped to the `text` field.
2. One or more `query` entries, mapped to the `query` field.
3. For each `query` entry, there will be the following dependent fields:
   * `doi`
   * `title` 
   * `full_text`
   * `bibtex`
   * `pdf_location`
   * `journal`
   * `citation_count`
   * `relevance_score`

To reconstruct the Excel table from the JSON, for each `point_text`, its value should be repeated for each associated `query`. This means that if there are 7 queries for a particular `point_text`, that `point_text` value should be repeated 7 times in the Excel table, with each repetition accompanied by the corresponding `query` and its dependent fields.

In [11]:
import os
import json
import pandas as pd

# Set the path to the documents folder
documents_folder = "documents"

# Create an Excel writer object
#output_file = documents_folder + "/master.xlsx"
#writer = pd.ExcelWriter(output_file, engine="openpyxl")

# Iterate over the JSON files in the documents folder
for filename in os.listdir(documents_folder):
    if filename.startswith("outline_") and filename.endswith(".json"):
        file_path = os.path.join(documents_folder, filename)
        
        # Extract the sheet name from the filename
        sheet_name = filename.split("_")[1].split(".")[0]
        
        # Load the JSON data from the file
        with open(file_path, "r") as file:
            json_data = json.load(file)
        
        # Create an empty list to store the data for the current JSON file
        data_list = []
        
        # Extract the data from the JSON structure
        for section in json_data:
            document_title = section["title"]
            for subsection in section.get("subsections", []):
                section_title = subsection["title"]
                for point_key, point_data in subsection.items():
                    if point_key.startswith("point"):
                        for query_key, query_data in point_data.items():
                            if query_key.startswith("query"):
                                data_list.append({
                                    "document_title": document_title,
                                    "section_title": section_title,
                                    "point_text": point_data["text"],
                                    "query": query_data,
                                    "doi": "",
                                    "title": "",
                                    "full_text": "",
                                    "bibtex": "",
                                    "pdf_location": "",
                                    "journal": "",
                                    "citation_count": "",
                                    "relevance_score": ""
                                })
        
        # Create a DataFrame from the data list
        df = pd.DataFrame(data_list)
        
        # Reorder the columns
        column_order = [
            "document_title",
            "section_title",
            "point_text",
            "query",
            "doi",
            "title",
            "full_text",
            "bibtex",
            "pdf_location",
            "journal",
            "citation_count",
            "relevance_score"
        ]
        df = df[column_order]
        
        # Save the DataFrame to a separate sheet in the Excel file
        df.to_excel(writer, sheet_name=sheet_name, index=False)

# Save the Excel file
writer.close()
print(f"Data from JSON files loaded and saved to {output_file}.")

Data from JSON files loaded and saved to documents/master.xlsx.


In [ ]:
import openpyxl
import asyncio
import json
from llm_api_handler import LLM_APIHandler
import nest_asyncio
nest_asyncio.apply()

from prompts import get_prompt, remove_illegal_characters, outline, review_intention

class ExcelQueryProcessor:
    def __init__(self, file_path, api_key_path):
        self.workbook = openpyxl.load_workbook(file_path)
        self.api_handler = LLM_APIHandler(api_key_path)

    async def process_queries(self, sheet_name, prompt_template, input_columns, output_columns):
        sheet = self.workbook[sheet_name]
        header_row = next(sheet.iter_rows(values_only=True))
        column_names = {cell: index for index, cell in enumerate(header_row)}
        
        query_data = []
        for row in sheet.iter_rows(values_only=True, min_row=2):
            print(f"Processing rows")
            print(f"Column names:{column_names}")
            try:
                if not row[column_names["full_text"]]:
                    print(f"Skipping row {row[column_names['unique_identifier']]} because it has no full text.")
                    continue
                query_values = {col: row[column_names[col]] for col in input_columns}
                print(f"Processing query: {query_values}")
                query_data.append({"unique_identifier": row[column_names["unique_identifier"]], "query": query_values})
            except Exception as e:
                print(f"Error occurred: {str(e)}")
                break

        results = await self.process_queries_async(query_data, prompt_template, output_columns)
        self.update_sheet(sheet, results, output_columns, column_names)
        self.workbook.save(file_path)

    async def process_queries_async(self, query_data, prompt_template, output_columns):
        tasks = [asyncio.ensure_future(self.process_single_query(query, prompt_template, output_columns)) for query in query_data]
        return await asyncio.gather(*tasks)

    async def process_single_query(self, query, prompt_template, output_columns):
        while True:
            prompt = get_prompt(prompt_template, outline=outline, review_intention=review_intention, **query["query"])
            response = await self.api_handler.generate_gemini_content(prompt)
            try:
                response_json = json.loads(response)
                return {"unique_identifier": query["unique_identifier"], "response": response_json}
            except json.JSONDecodeError:
                print(f"Invalid JSON response. Retrying query: {prompt}")

    def update_sheet(self, sheet, results, output_columns, column_names):
        for col in output_columns:
            if col not in column_names:
                sheet.cell(row=1, column=sheet.max_column + 1, value=col)
                column_names[col] = sheet.max_column

        for result in results:
            unique_identifier = result["unique_identifier"]
            response = result["response"]
            row = [r for r in sheet.iter_rows() if r[column_names["unique_identifier"]].value == unique_identifier][0]
            for col, value in response.items():
                if col in output_columns:
                    cleaned_value = remove_illegal_characters(value)
                    sheet.cell(row=row[0].row, column=column_names[col], value=cleaned_value)

# Usage example
file_path = "documents/master.xlsx"
api_key_path = r"C:\Users\bnsoh2\OneDrive - University of Nebraska-Lincoln\Documents\keys\api_keys.json"
sheet_name = "1"
prompt_template = "paper_analysis"
input_columns = ["point_text", "section_title", "document_title", "full_text", "unique_identifier"]
output_columns = ["relevance_score", "explanation", "extract_1", "extract_2", "Alternate_section"]

processor = ExcelQueryProcessor(file_path, api_key_path)
asyncio.run(processor.process_queries(sheet_name, prompt_template, input_columns, output_columns))

Processing rows
Column names:{'point_text': 0, 'section_title': 1, 'document_title': 2, 'query': 3, 'doi': 4, 'title': 5, 'full_text': 6, 'bibtex': 7, 'pdf_location': 8, 'journal': 9, 'citation_count': 10, 'relevance_score': 11, 'unique_identifier': 12, 'explanation': 13, 'extract_1': 14, 'extract_2': 15, 'Alternate_section': 16}
Processing query: {'point_text': 'â€¢ The challenge of feeding a growing population with finite resources', 'section_title': '1.1 The Global Food Challenge and the Role of Irrigation', 'document_title': '1. Introduction', 'full_text': 'The Challenge of Feeding the World\nby Dániel Fróna 1,2,*,János Szenderák 1,2 andMónika Harangi-Rákos 2\n1\nKaroly Ihrig Doctoral School of Management and Business, University of Debrecen, 4032 Debrecen, Hungary\n2\nInstitute of Sectoral Economics and Methodology, University of Debrecen, 4032 Debrecen, Hungary\n*\nAuthor to whom correspondence should be addressed.\nSustainability 2019, 11(20), 5816; https://doi.org/10.3390/su112

ERROR:root:Error in Gemini API call: Invalid JSON format in the response


{
 "explanation": "This research document does not relate to the specific point of \u201cthe challenge of feeding a growing population with finite resources\u201d in the context of automated irrigation systems. Therefore, an analysis of relevance based on the context provided is not possible.",
 "extract_1": null,
 "extract_2": null,
 "relevance_score": 0.0,
 "Alternate_section": ""
}


ValueError: Invalid JSON format in the response

ERROR:root:Error in Gemini API call: Invalid JSON format in the response
ERROR:root:Error in Gemini API call: Invalid JSON format in the response


...

{
    "explanation": "I apologize, but the provided text does not contain the specific point of discussion you are looking for. The text focuses on the \"Global Food Challenge and the Role of Irrigation\" and does not discuss the given point on \"the necessity of scalable water-efficient practices for increasing food demand\". Unfortunately, I cannot provide an accurate analysis without the relevant information in the context.",
    "extract_1": null,
    "extract_2": null,
    "relevance_score": 0.0,
    "Alternate_section": null
}


ERROR:root:Error in Gemini API call: Invalid JSON format in the response


{
 "explanation": "The provided context does not mention the paper in question, so I cannot provide an analysis of its relevance to the point in question.",
 "extract_1": null,
 "extract_2": null,
 "relevance_score": 0.0,
 "Alternate_section": ""
}


ERROR:root:Error in Gemini API call: Invalid JSON format in the response
ERROR:root:Error in Gemini API call: Invalid JSON format in the response


{
 "explanation": "I apologize, but the provided full text is inaccessible, and therefore, I cannot perform an in-depth analysis of the paper's relevance to the specific point provided. Without access to the full text, I am unable to provide you with a detailed explanation, relevant quotes, or a relevance score.",
 "extract_1": null,
 "extract_2": null,
 "relevance_score": 0,
 "Alternate_section": ""
}
{
 "explanation": "The reference document does not contain any relevant information regarding the definition of irrigation scheduling and management. Therefore, I cannot provide you with any extracted quotes or a relevance score.",
 "extract_1": null,
 "extract_2": null,
 "relevance_score": 0.0,
 "Alternate_section": null
}
